In [22]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime
import calendar
import json
import csv
import numpy as np
from selenium import webdriver
import time
import math
import re
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
import os
import pickle
from pycaret.regression import setup, compare_models, evaluate_model, predict_model, save_model,load_model, tune_model, pull, get_metrics, finalize_model, get_config, create_model
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from pycaret.utils import check_metric
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

# Preparation

In [17]:
gas_stations = pd.read_csv('[HUN_FUEL]03_28.csv')

In [18]:
def get_region(zip_code):
    if 1011 <= zip_code <= 1806:
        return 'Budapest'
    elif 2000 <= zip_code <= 2769:
        return 'Pest megye'
    elif (2400 <= zip_code <= 2490) or (8000 <= zip_code <= 8157):
        return 'Fejér megye'
    elif (2500 <= zip_code <= 2545) or (2800 <= zip_code <= 2949):
        return 'Komárom-Esztergom megye'
    elif (2640 <= zip_code <= 2699) or (3041 <= zip_code <= 3253):
        return 'Nógrád megye'
    elif (3000 <= zip_code <= 3036) or (3200 <= zip_code <= 3399):
        return 'Heves megye'
    elif 3400 <= zip_code <= 3999:
        return 'Borsod-Abaúj-Zemplén megye'
    elif 4000 <= zip_code <= 4288:
        return 'Hajdú-Bihar megye'
    elif (4300 <= zip_code <= 4977):
        return 'Szabolcs-Szatmár-Bereg megye'
    elif (5000 <= zip_code <= 5476):
        return 'Jász-Nagykun-Szolnok megye'
    elif (5500 <= zip_code <= 5948):
        return 'Békés megye'
    elif (6000 <= zip_code <= 6528):
        return 'Bács-Kiskun megye'
    elif (6600 <= zip_code <= 6932):
        return 'Csongrád vármegye'
    elif (7020 <= zip_code <= 7228):
        return 'Tolna megye'
    elif (7300 <= zip_code <= 7396) or (7600 <= zip_code <= 7985):
        return 'Baranya megye'
    elif (7400 <= zip_code <= 7589) or (8600 <= zip_code <= 8739):
        return 'Somogy vármegye'
    elif 8161 <= zip_code <= 8598:
        return 'Veszprém megye'
    elif (8353 <= zip_code <= 8395) or (8741 <= zip_code <= 8999):
        return 'Zala megye'
    elif 9001 <= zip_code <= 9495:
        return 'Győr-Moson-Sopron megye'
    elif 9500 <= zip_code <= 9985:
        return 'Vas megye'
    else:
        return math.nan

gas_stations['Services'] = gas_stations['Services'].str.split(';').str.len()
gas_stations['County'] = gas_stations['Post Code'].apply(get_region)
gas_stations = gas_stations[['City', 'County', 'Franchise (or private)', 'Highway', 'Services','Number of competition 1km', 'Number of competition 5km', 
        'Number of competition 10km','Number of competition 20km', 'Number of competition 30km', 'Number of competition 40km', 'Housing', 'Retail stores', 
        'Unemployed 15-64', 'Allowance', 'Income tax per person', 'Commuters Abroad','N drinking water', 'Employed 15-64', 'Tax per 1000', 'N of cars per 1000',
        'N of agriculture', 'Commuter Difference','Income Tax per Person', 'Locally Employed', '95-ös Benzin E10', 'Gázolaj', 'Lpg', '100-As Benzin E5', 'Cng', 
        'Prémium Gázolaj', 'Prémium Benzin E10','98-As Benzin E5', 'Adblue', 'Type-1 Elektromos Töltő','Type-2 Elektromos Töltő', 'Chademo Elektromos Töltő',
       'Ccs - Elektromos Töltő']]

# On which attributes is Regression Doable

In [19]:
target_variables_for_check = gas_stations[['95-ös Benzin E10', 'Gázolaj', 'Lpg', '100-As Benzin E5', 'Cng', 
        'Prémium Gázolaj', 'Prémium Benzin E10','98-As Benzin E5', 'Adblue', 'Type-1 Elektromos Töltő','Type-2 Elektromos Töltő', 'Chademo Elektromos Töltő',
       'Ccs - Elektromos Töltő']]

target_variables_for_check.count()

95-ös Benzin E10            1120
Gázolaj                     1128
Lpg                          287
100-As Benzin E5             932
Cng                            0
Prémium Gázolaj              894
Prémium Benzin E10           297
98-As Benzin E5               25
Adblue                        91
Type-1 Elektromos Töltő        1
Type-2 Elektromos Töltő        3
Chademo Elektromos Töltő       2
Ccs - Elektromos Töltő         1
dtype: int64

This shows that 95-ös Benzin E10, Gázolaj, 100-As Benzin E5, Prémium Gázolaj are the variables that we are going to build regression models with.

In [20]:
gas_stations = gas_stations[['City', 'County', 'Franchise (or private)', 'Highway', 'Services','Number of competition 1km', 'Number of competition 5km', 
        'Number of competition 10km','Number of competition 20km', 'Number of competition 30km', 'Number of competition 40km', 'Housing', 'Retail stores', 
        'Unemployed 15-64', 'Allowance', 'Income tax per person', 'Commuters Abroad','N drinking water', 'Employed 15-64', 'Tax per 1000', 'N of cars per 1000',
        'N of agriculture', 'Commuter Difference','Income Tax per Person', 'Locally Employed', '95-ös Benzin E10', 'Gázolaj', '100-As Benzin E5', 
        'Prémium Gázolaj']]

# Regression

In [26]:
attributes = ['City', 'County', 'Franchise (or private)', 'Highway', 'Services', 'Number of competition 1km', 'Number of competition 5km', 'Number of competition 10km', 'Number of competition 20km', 'Number of competition 30km', 'Number of competition 40km', 'Housing', 'Retail stores', 'Unemployed 15-64', 'Allowance', 'Income tax per person', 'Commuters Abroad', 'N drinking water', 'Employed 15-64', 'Tax per 1000', 'N of cars per 1000', 'N of agriculture', 'Commuter Difference', 'Income Tax per Person', 'Locally Employed']
targets = ['95-ös Benzin E10', 'Gázolaj', '100-As Benzin E5', 'Prémium Gázolaj']

regression_results_list = []
tuned_best_performances = []

for target_local in targets:
    
    gas_stations_local = gas_stations[attributes + [target_local]]
    gas_stations_local = gas_stations_local.dropna()

    path = '/home/csaba/Desktop/Gas Competition Analysis/Saved Models'
    os.chdir(path)

    train_data, test_data = train_test_split(gas_stations_local, test_size=0.2, random_state=42)
    
    categorical_features = ['City', 'County', 'Franchise (or private)', 'Highway']
    numeric_features = ['Number of competition 1km', 'Number of competition 5km', 'Number of competition 10km', 'Number of competition 20km', 'Number of competition 30km', 'Number of competition 40km', 'Housing', 'Retail stores', 'Unemployed 15-64', 'Allowance', 'Income tax per person', 'Commuters Abroad', 'N drinking water', 'Employed 15-64', 'Tax per 1000', 'N of cars per 1000', 'N of agriculture', 'Commuter Difference', 'Income Tax per Person', 'Locally Employed']
    
    s = setup(train_data, target=target_local, numeric_features=numeric_features)

    best_model = compare_models()

    regression_results = pull()

    regression_results_list.append(regression_results)

    tuned_best_model = tune_model(best_model, optimize = 'R2')

    final_model = finalize_model(tuned_best_model)

    # get performance metrics on the test set

    X_test = test_data.drop([target_local], axis=1)

    y_test = test_data[target_local].to_numpy()

    y_pred = predict_model(final_model, data=X_test)['Label'].to_numpy()

    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)

    performance_metrics = {"mse": mse, "r2": r2, "mae": mae}

    # Append the performance metrics to tuned_best_performances
    tuned_best_performances.append(performance_metrics)

    # append the performance metrics to tuned_best_performances
    tuned_best_performances.append(performance_metrics)

    # save the models
    save_model(final_model, f'{target_local}')

    path = '/home/csaba/Desktop/Gas Competition Analysis'
    os.chdir(path)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,1.9664,24.2004,4.7387,0.6869,0.0078,0.0033,0.2480
gbr,Gradient Boosting Regressor,2.1501,24.8382,4.7908,0.6837,0.0079,0.0036,0.1190
et,Extra Trees Regressor,1.7073,23.4392,4.6541,0.6815,0.0077,0.0028,0.2360
xgboost,Extreme Gradient Boosting,2.2248,29.6255,5.1637,0.6332,0.0085,0.0037,0.9280
br,Bayesian Ridge,2.9876,30.0457,5.2940,0.6171,0.0087,0.0049,0.0560
ridge,Ridge Regression,3.1239,31.5735,5.4306,0.5979,0.0089,0.0052,0.0150
omp,Orthogonal Matching Pursuit,3.0027,34.6116,5.5917,0.5762,0.0092,0.0050,0.0200
lightgbm,Light Gradient Boosting Machine,3.1905,34.1430,5.6169,0.5683,0.0093,0.0053,0.1940
dt,Decision Tree Regressor,2.0694,34.8593,5.7864,0.5357,0.0096,0.0034,0.0170
ada,AdaBoost Regressor,4.7098,43.5112,6.4170,0.4338,0.0106,0.0078,0.0460


IntProgress(value=0, description='Processing: ', max=7)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:40:44
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Regressor


,Fold,MAE,MSE,RMSE,R2,RMSLE,MAPE


Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    7.2s


KeyboardInterrupt: 

# Evaluation

In [7]:
targets = ['95-ös Benzin E10', 'Gázolaj', '100-As Benzin E5', 
        'Prémium Gázolaj']
i = 0
pd.options.display.float_format = "{:,.2f}".format
for df in regression_results_list:
    print(targets[i] + ' Regressor Performances:')
    i += 1
    display(df.iloc[:-1])

95-ös Benzin E10 Regressor Performances:


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,2.37,28.05,5.19,0.69,0.01,0.00,0.11
br,Bayesian Ridge,3.05,29.71,5.38,0.67,0.01,0.01,0.04
rf,Random Forest Regressor,2.10,30.13,5.37,0.67,0.01,0.00,0.25
ridge,Ridge Regression,3.16,30.35,5.44,0.66,0.01,0.01,0.01
lightgbm,Light Gradient Boosting Machine,3.11,31.56,5.52,0.65,0.01,0.01,0.18
omp,Orthogonal Matching Pursuit,3.08,33.07,5.68,0.64,0.01,0.01,0.01
et,Extra Trees Regressor,1.92,32.62,5.53,0.63,0.01,0.00,0.24
xgboost,Extreme Gradient Boosting,2.32,34.18,5.57,0.61,0.01,0.00,0.30
dt,Decision Tree Regressor,2.24,44.26,6.43,0.50,0.01,0.00,0.02
ada,AdaBoost Regressor,5.24,47.61,6.85,0.47,0.01,0.01,0.05


Gázolaj Regressor Performances:


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,2.56,36.87,5.67,0.50,0.01,0.00,0.12
rf,Random Forest Regressor,2.15,36.53,5.60,0.49,0.01,0.00,0.33
br,Bayesian Ridge,3.41,41.02,6.10,0.44,0.01,0.01,0.08
ridge,Ridge Regression,3.39,40.66,6.06,0.43,0.01,0.01,0.01
omp,Orthogonal Matching Pursuit,2.84,39.50,5.92,0.43,0.01,0.00,0.02
et,Extra Trees Regressor,2.10,41.45,5.99,0.38,0.01,0.00,0.34
lightgbm,Light Gradient Boosting Machine,3.54,44.39,6.34,0.37,0.01,0.01,0.04
xgboost,Extreme Gradient Boosting,2.62,42.05,6.10,0.36,0.01,0.00,0.31
dt,Decision Tree Regressor,2.17,42.53,6.14,0.36,0.01,0.00,0.02
ada,AdaBoost Regressor,5.07,62.34,7.70,0.08,0.01,0.01,0.07


100-As Benzin E5 Regressor Performances:


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,1.91,25.62,4.74,0.93,0.01,0.00,0.21
gbr,Gradient Boosting Regressor,2.09,27.19,4.89,0.93,0.01,0.00,0.09
ridge,Ridge Regression,2.84,28.09,5.15,0.93,0.01,0.00,0.01
et,Extra Trees Regressor,1.73,29.33,4.98,0.93,0.01,0.00,0.17
omp,Orthogonal Matching Pursuit,2.41,27.77,5.08,0.93,0.01,0.00,0.02
br,Bayesian Ridge,2.86,28.43,5.18,0.93,0.01,0.00,0.03
dt,Decision Tree Regressor,1.81,30.26,5.19,0.92,0.01,0.00,0.01
xgboost,Extreme Gradient Boosting,2.18,34.17,5.60,0.91,0.01,0.00,0.24
lightgbm,Light Gradient Boosting Machine,3.42,35.83,5.84,0.91,0.01,0.01,0.03
ada,AdaBoost Regressor,4.73,44.52,6.54,0.88,0.01,0.01,0.05


Prémium Gázolaj Regressor Performances:


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,1.62,31.98,5.39,0.82,0.01,0.00,0.19
rf,Random Forest Regressor,2.05,39.56,5.84,0.78,0.01,0.00,0.20
gbr,Gradient Boosting Regressor,2.43,42.13,6.08,0.77,0.01,0.00,0.09
br,Bayesian Ridge,3.58,44.07,6.42,0.75,0.01,0.01,0.05
ridge,Ridge Regression,3.58,44.30,6.44,0.75,0.01,0.01,0.01
dt,Decision Tree Regressor,2.00,52.56,6.42,0.71,0.01,0.00,0.01
xgboost,Extreme Gradient Boosting,2.45,54.55,6.77,0.70,0.01,0.00,0.24
lightgbm,Light Gradient Boosting Machine,4.12,52.73,7.06,0.70,0.01,0.01,0.03
omp,Orthogonal Matching Pursuit,3.03,54.24,7.09,0.70,0.01,0.00,0.01
ada,AdaBoost Regressor,6.72,81.97,8.97,0.53,0.01,0.01,0.06


In [8]:
best_model_performances = pd.DataFrame(tuned_best_performances)
best_model_performances['rmse'] = np.sqrt(best_model_performances['mse'])
best_model_performances = best_model_performances.iloc[::2].reset_index(drop=True)
best_model_performances.index = targets
best_model_performances

,mse,r2,mae,rmse
95-ös Benzin E10,23.64,0.75,2.79,4.86
Gázolaj,15.22,0.76,1.89,3.90
100-As Benzin E5,25.42,0.93,1.18,5.04
Prémium Gázolaj,26.67,0.82,1.59,5.16


In [14]:
gas_stations['Prémium Gázolaj'].dropna().describe()

count   894.00
mean    661.37
std      12.18
min     605.00
25%     661.00
50%     661.00
75%     661.90
max     697.90
Name: Prémium Gázolaj, dtype: float64

**95 Benzin E10**:

    - Mean Squared Error (MSE) = 23.64
    - R-squared (R2) = 0.75
    - Mean Absolute Error (MAE) = 2.79
    - Root Mean Squared Error (RMSE) = 4.86

To determine if these metrics indicate a good result, you need to compare them with the range of values of your target variable, which is "count" in this case. You can see that the range of "count" is from 569 to 647 with a mean of 605.96 and a standard deviation of 9.02.

Comparing the RMSE with the standard deviation of the target variable, you can see that the RMSE is less than half of the standard deviation. This is usually considered a good result, indicating that the model's predictions are relatively close to the true values.

The R-squared value of 0.75 indicates that the model explains 75% of the variance in the target variable. This is a decent score, but it's worth noting that the remaining 25% of the variance is unexplained.

**Gázolaj**:

    - Mean Squared Error (MSE) = 15.22
    - R-squared (R2) = 0.76
    - Mean Absolute Error (MAE) = 1.89
    - Root Mean Squared Error (RMSE) = 3.90

To determine if these metrics indicate a good result, you need to compare them with the range of values of your target variable, which is "count" in this case. You can see that the range of "count" is from 579.90 to 689.90 with a mean of 606.05 and a standard deviation of 8.59.

Comparing the RMSE with the standard deviation of the target variable, you can see that the RMSE is less than half of the standard deviation. This is usually considered a good result, indicating that the model's predictions are relatively close to the true values.

The R-squared value of 0.76 indicates that the model explains 76% of the variance in the target variable. This is a decent score, but it's worth noting that the remaining 24% of the variance is unexplained.

**100-as Benzin E5**

    - Mean Squared Error (MSE) = 25.42
    - R-squared (R2) = 0.93
    - Mean Absolute Error (MAE) = 1.18
    - Root Mean Squared Error (RMSE) = 5.04

To determine if these metrics indicate a good result, you need to compare them with the range of values of your target variable, which is "count" in this case. You can see that the range of "count" is from 598.90 to 728.00 with a mean of 651.30 and a standard deviation of 19.25.

Comparing the RMSE with the standard deviation of the target variable, you can see that the RMSE is slightly more than a quarter of the standard deviation. This is usually considered a good result, indicating that the model's predictions are relatively close to the true values.

The R-squared value of 0.93 indicates that the model explains 93% of the variance in the target variable. This is a high score, indicating that the model is doing a good job of fitting the data.

**Prémium Gázolaj**: Based on the evaluation metrics and the range of values for the target variable, it seems like your model has produced a decent result, but there is definitely room for improvement. Whether the result is good enough depends on the specific context and requirements of your problem.